# Mean Field Approximation and Coordinate Ascent 

In this notebook, we consider the Ising model on the 10*10 lattice such that $ P(x) = \frac{1}{Z}\prod_{i>j} \phi (x_{i}, x_{j}) $ and $ \phi (x_{i}, x_{j}) = exp (\beta \: I[ x_{i} = x_{j} ]$ for i a neighbour of j on the lattice and i>j to avoid overcounting. \\

We aim to compute the joint probability distribution of the top and bottom nodes of the rightmost column of the lattice for different values of beta. \\

We use mean field approximation and coordinate ascent. 

In [12]:
import sys, os
import random
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# Step 2 : define the relationship between the nodes 

In [13]:
def get_mn(k):
    if k < xmax:
        m = k
        n = 0 
    if k > xmax-1:
        m = k % 10
        n = (k-m)/10
    return m,n

def get_k(m,n):
    k = m+(n*10)
    return k

# Step 3 : Check overflow issues 

In [14]:
#check whether index is within the 10*10 lattice 

def check_overflow(i, j):
    if (0 <= i < xmax) and (0 <= j < ymax) :
        return i,j
    elif (0 > i > xmax) and (0 > j > ymax):
        return ('overflow issue')

# Step 4 : Get the neighbours of node k

In [15]:
def neighbours(k):
    
    #define x and y
    x, y = get_mn(k)
    
    #i
    ineig = [(x+1,y),(x,y+1)]
    i_neigh_list = [check_overflow(i_neighb, j_neighb) for (i_neighb, j_neighb) in ineig]
    i = list(filter(None,i_neigh_list))
    
    for n in range(len(i)):
        x_ = i[n][0]
        y_ = i[n][1]
        i[n] = get_k(x_, y_)
    
    #j
    jneig = [(x-1,y),(x,y-1)]
    j_neigh_list = [check_overflow(i_neighb, j_neighb) for (i_neighb, j_neighb) in jneig]
    j = list(filter(None,j_neigh_list))
    
    for n in range(len(j)):
        x_ = j[n][0]
        y_ = j[n][1]
        j[n] = get_k(x_, y_)
        
    return i,j


# Step 5 : Update parameter theta of the Bernouilli

Let us recall that the update of the parameter theta is made using logistic regression and the sigmoid function

In [16]:
def sigmoid(x):
    sigmoid = 1/(1 + np.exp(-x))
    return sigmoid

def update_theta(theta,k):
    
    i_= 0
    j_= 0
    
    isimk, ksimj = neighbours(k) 
    
    ik_list = [int(i) for i in isimk]
    kj_list = [int(k) for k in ksimj]
    
    for i in ik_list:
        i_ += 1-2*rdm[i]
    
    for j in kj_list:
        j_ += 1-2*rdm[j]
    
    update = sigmoid(((i_+j_)*theta)*-1 )
        
    return update

# Step 5 : Define ELBO

Let us recall that the ELBO is the sum on the loglikelihood and the entropy functions (as proved in the mathematical demonstration)

In [17]:
def entropy(k): 
    entropy = np.log(1-rdm[k]) * (1-rdm[k]) + np.log(rdm[k]) * rdm[k] * -1
    return entropy 

def logLk(theta,k):
    
    i_ = 0 
    j_ = 0 
    
    isimk, ksimj = neighbours(k) 
    ik_list = [int(i) for i in isimk]
    kj_list = [int(k) for k in ksimj]

    for i in ik_list:
        i_ += rdm[i]*rdm[k]+(1-rdm[i])*(1-rdm[k])

    for j in kj_list:
        j_ += rdm[k]*rdm[j]+(1-rdm[k])*(1-rdm[j])
        
    loglk = theta*(i_+j_) 
        
    return loglk

def ELBO(theta): 
    
    elbo = 0
    
    for k in range(xmax*ymax):
        
        elbo += logLk(beta,k) + entropy(k) #update elbo
        isimk, ksimj = neighbours(k) #update neighours of node k 
        rdm[k] = update_theta(theta,k) #update theta
        
    return elbo

# Define parameters and get probability tables for beta = 0.01

In [26]:
beta = 0.01

#Define the lattice 
xmax = 10
ymax = 10

#pre-compute : less time consumming 
rdm = np.random.random(xmax * ymax)

tends_to_0 = 0.000001
tends_to_inf = 100000

#initialization 
update_elbo =0

while 1:
    update_elbo = ELBO(beta)
    
    if tends_to_inf - update_elbo < tends_to_0 and tends_to_inf - update_elbo >= 0:
        break
        
    tends_to_inf = update_elbo
    
X_1_10_is_1= rdm[90] 
X_1_10_is_0= 1-rdm[90]
X_10_10_is_1= rdm[99]
X_10_10_is_0= 1-rdm[99]

print(X_1_10_is_0*X_10_10_is_0)
print(X_1_10_is_0*X_10_10_is_1)
print(X_1_10_is_1*X_10_10_is_0)
print(X_1_10_is_1*X_10_10_is_1)

0.2500000000000027
0.2500000000000027
0.24999999999999728
0.24999999999999728


# Define parameters and get probability tables for beta = 1

In [29]:
beta = 1

#Define the lattice 
xmax = 10
ymax = 10

#pre-compute : less time consumming 
rdm = np.random.random(xmax * ymax)

tends_to_0 = 0.000001
tends_to_inf = 100000

#initialization 
update_elbo =0

while 1:
    update_elbo = ELBO(beta)
    
    if tends_to_inf - update_elbo < tends_to_0 and tends_to_inf - update_elbo >= 0:
        break
        
    tends_to_inf = update_elbo
    
X_1_10_is_1= rdm[90] 
X_1_10_is_0= 1-rdm[90]
X_10_10_is_1= rdm[99]
X_10_10_is_0= 1-rdm[99]

print(X_1_10_is_0*X_10_10_is_0)
print(X_1_10_is_0*X_10_10_is_1)
print(X_1_10_is_1*X_10_10_is_0)
print(X_1_10_is_1*X_10_10_is_1)

0.131927020395963
0.7134737097478534
0.024125625044170526
0.13047364481201296


# Define parameters and get probability tables for beta = 4

In [25]:
beta = 4

#Define the lattice 
xmax = 10
ymax = 10

#pre-compute : less time consumming 
rdm = np.random.random(xmax * ymax)

tends_to_0 = 0.000001
tends_to_inf = 100000

#initialization 
update_elbo =0

while 1:
    update_elbo = ELBO(beta)
    
    if tends_to_inf - update_elbo < tends_to_0 and tends_to_inf - update_elbo >= 0:
        break
        
    tends_to_inf = update_elbo
    
X_1_10_is_1= rdm[90] 
X_1_10_is_0= 1-rdm[90]
X_10_10_is_1= rdm[99]
X_10_10_is_0= 1-rdm[99]

print(X_1_10_is_0*X_10_10_is_0)
print(X_1_10_is_0*X_10_10_is_1)
print(X_1_10_is_1*X_10_10_is_0)
print(X_1_10_is_1*X_10_10_is_1)

1.1248187637422915e-07
0.0003352706965542518
0.0003352706965542518
0.9993293461250151
